# Libs and dataset import

In [ ]:
import nltk
nltk.download('all')

In [ ]:
# %%bash
# cd '/content/drive/MyDrive/Colab Notebooks/ressources/la beubli/treetagger'
# # Download the tagger package for your system (PC-Linux, Mac OS-X, ARM64, ARMHF, ARM-Android, PPC64le-Linux).
# wget https://cis.lmu.de/~schmid/tools/TreeTagger/data/tree-tagger-linux-3.2.4.tar.gz
# tar -xzvf tree-tagger-linux-3.2.4.tar.gz
# # Download the tagging scripts into the same directory.
# wget https://cis.lmu.de/~schmid/tools/TreeTagger/data/tagger-scripts.tar.gz
# gunzip tagger-scripts.tar.gz
# # Download the installation script install-tagger.sh.
# wget https://cis.lmu.de/~schmid/tools/TreeTagger/data/install-tagger.sh
# # Download the parameter files for the languages you want to process.
# # list of all files (parameter files) https://cis.lmu.de/~schmid/tools/TreeTagger/#parfiles
# wget https://cis.lmu.de/~schmid/tools/TreeTagger/data/english.par.gz

# wget https://cis.lmu.de/~schmid/tools/TreeTagger/data/french.par.gz

# sh install-tagger.sh
# cd ..
# sudo pip install treetaggerwrapper

In [ ]:
# !pip install --upgrade treetaggerwrapper
!pip install googletrans==4.0.0-rc1

In [ ]:
#load libs
from google.colab import files
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
np.warnings.filterwarnings('ignore')
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup
import re
import pprint   # For proper print of sequences.
# import treetaggerwrapper
from googletrans import Translator
from nltk.tokenize import sent_tokenize 
import glob

In [ ]:
complete_french_bible = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ressources/la beubli/complete_french_bible/complete_bible.csv')
complete_french_bible

,Unnamed: 0,chapter_number,page_number,line_number,sentence
0,0,1.0,1.0,1,"Au commencement, Dieu créa les cieux et la terre."
1,1,1.0,1.0,2,La terre était informe et vide: il y avait des...
2,2,1.0,1.0,3,Dieu dit: Que la lumière soit! Et la lumière fut.
3,3,1.0,1.0,4,Dieu vit que la lumière était bonne; et Dieu s...
4,4,1.0,1.0,5,"Dieu appela la lumière jour, et il appela les ..."
...,...,...,...,...,...
31097,31097,66.0,22.0,17,Et l'Esprit et l'épouse disent: Viens. Et que ...
31098,31098,66.0,22.0,18,Je le déclare à quiconque entend les paroles d...
31099,31099,66.0,22.0,19,et si quelqu'un retranche quelque chose des pa...
31100,31100,66.0,22.0,20,"Celui qui atteste ces choses dit: Oui, je vien..."


# Sentence tokenization

We see that the sentence feature is varied in length, from a few words to several long sentences. 

Before translation, we need to cut into simple sentences.


In [ ]:
def split_line_into_sentences(text_line):
  sentence_list = sent_tokenize(text_line, language='french')
  sentence_list = np.array(sentence_list)
  sentence_list.resize(8)
  return sentence_list



# code that allowed to choose 8
# as the max number of sentences per text line

# max_len = 0
# for sentence in tqdm(complete_french_bible.sentence):
#   sentences = split_line_into_sentences(sentence)
#   if (len(sentences) > max_len):
#     max_len = len(sentences)
#   # break

In [ ]:
#all of this code splits text lines into sentences and pivots longs to add a new column called "sentence number"

sentence_dataset = pd.concat([complete_french_bible[['chapter_number', 'page_number', 'line_number']], complete_french_bible.apply(lambda row: split_line_into_sentences(row['sentence']), axis=1, result_type='expand')], axis=1)
sentence_dataset.columns = sentence_dataset.columns.map(str)
sentence_dataset= pd.melt(sentence_dataset, id_vars=['chapter_number', 'page_number', 'line_number'], value_vars=['0', '1', '2', '3', '4', '5', '6', '7'], var_name='sentence_number', value_name='sentence')
sentence_dataset.sentence_number = sentence_dataset.sentence_number.astype(int)
sentence_dataset.sentence = sentence_dataset.sentence.astype(str)
sentence_dataset = sentence_dataset[sentence_dataset.sentence != '']
sentence_dataset = sentence_dataset.sort_values(['chapter_number','page_number','line_number','sentence_number'])
sentence_dataset.reset_index(drop=True, inplace=True)
sentence_dataset

,chapter_number,page_number,line_number,sentence_number,sentence
0,1.0,1.0,1,0,"Au commencement, Dieu créa les cieux et la terre."
1,1.0,1.0,2,0,La terre était informe et vide: il y avait des...
2,1.0,1.0,3,0,Dieu dit: Que la lumière soit!
3,1.0,1.0,3,1,Et la lumière fut.
4,1.0,1.0,4,0,Dieu vit que la lumière était bonne; et Dieu s...
...,...,...,...,...,...
40855,66.0,22.0,19,0,et si quelqu'un retranche quelque chose des pa...
40856,66.0,22.0,20,0,"Celui qui atteste ces choses dit: Oui, je vien..."
40857,66.0,22.0,20,1,Amen!
40858,66.0,22.0,20,2,"Viens, Seigneur Jésus!"


In [ ]:
sentence_dataset

,chapter_number,page_number,line_number,sentence_number,sentence
0,1.0,1.0,1,0,"Au commencement, Dieu créa les cieux et la terre."
1,1.0,1.0,2,0,La terre était informe et vide: il y avait des...
2,1.0,1.0,3,0,Dieu dit: Que la lumière soit!
3,1.0,1.0,3,1,Et la lumière fut.
4,1.0,1.0,4,0,Dieu vit que la lumière était bonne; et Dieu s...
...,...,...,...,...,...
40855,66.0,22.0,19,0,et si quelqu'un retranche quelque chose des pa...
40856,66.0,22.0,20,0,"Celui qui atteste ces choses dit: Oui, je vien..."
40857,66.0,22.0,20,1,Amen!
40858,66.0,22.0,20,2,"Viens, Seigneur Jésus!"


## Save sentence dataset to csv

In [ ]:
sentence_dataset.to_csv('/content/drive/MyDrive/Colab Notebooks/ressources/la beubli/sentence_dataset/sentence_dataset.csv')

# Translation -- TODO

In [ ]:
sentence_dataset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ressources/la beubli/sentence_dataset/sentence_dataset.csv', index_col=0)

In [ ]:
sentence_dataset

,chapter_number,page_number,line_number,sentence_number,sentence
0,1.0,1.0,1,0,"Au commencement, Dieu créa les cieux et la terre."
1,1.0,1.0,2,0,La terre était informe et vide: il y avait des...
2,1.0,1.0,3,0,Dieu dit: Que la lumière soit!
3,1.0,1.0,3,1,Et la lumière fut.
4,1.0,1.0,4,0,Dieu vit que la lumière était bonne; et Dieu s...
...,...,...,...,...,...
40855,66.0,22.0,19,0,et si quelqu'un retranche quelque chose des pa...
40856,66.0,22.0,20,0,"Celui qui atteste ces choses dit: Oui, je vien..."
40857,66.0,22.0,20,1,Amen!
40858,66.0,22.0,20,2,"Viens, Seigneur Jésus!"


## Testing bulk translation

The API wrapper bulk translation is dead it seems. We will have to make our own sample batches and backup results ourselves.

In [ ]:
def translate_string(input_string):
  translator = Translator()
  translation = translator.translate(input_string)
  return translation.text

translate_string('ceci est un test')

'this is a test'

# Using Google limit of 30k chars per day, we would need 135 days to process the complete sentences

In [ ]:
sentence_dataset.sentence.str.len().sum()

4066009

In [ ]:
sentence_dataset.sentence.str.len().sum()/30000

135.53363333333334

### What if we filter on dangerous sentences ?

"il" and "Il" containing sentences only represent half the words.  
Still a lot, but half.

In [ ]:
sentence_dataset[sentence_dataset.sentence.str.contains(r'(il|Il )')].sentence.str.len().sum()

2269253

In [ ]:
sentence_dataset[sentence_dataset.sentence.str.contains(r'(il|Il )')]

,chapter_number,page_number,line_number,sentence_number,sentence
1,1.0,1.0,2,0,La terre était informe et vide: il y avait des...
5,1.0,1.0,5,0,"Dieu appela la lumière jour, et il appela les ..."
6,1.0,1.0,5,1,"Ainsi, il y eut un soir, et il y eut un matin:..."
7,1.0,1.0,6,0,Dieu dit: Qu'il y ait une étendue entre les ea...
8,1.0,1.0,7,0,"Et Dieu fit l'étendue, et il sépara les eaux q..."
...,...,...,...,...,...
40842,66.0,22.0,10,0,Et il me dit: Ne scelle point les paroles de l...
40844,66.0,22.0,11,0,"Que celui qui est injuste soit encore injuste,..."
40847,66.0,22.0,14,0,"Heureux ceux qui lavent leurs robes, afin d'av..."
40850,66.0,22.0,16,1,"Je suis le rejeton et la postérité de David, l..."


## Small sentence extract to test Yandex file translate GUI - FAIL

It worked!

EDIT: limit unknown, impractical

In [ ]:
# sentence_dataset.head(100).sentence.to_csv('/content/small_export.csv')

### Go big or go home

Full extract to test the limits


EDIT: Go home, the files are not accepted anymore. We will split in 4 to test

In [ ]:
# sentence_dataset.sample(frac=0.10).sentence.to_csv('/content/tenth_export.csv')

# Analyze "il/Il" sentences

20k sentences containing the "il" chars (case insensitive).

In [ ]:
sentence_dataset_il = sentence_dataset[sentence_dataset.sentence.str.contains('il y était', case=False)]
sentence_dataset_il = sentence_dataset_il[['sentence'	]]

sentence_dataset_il

,sentence
13305,Il y était dit:
16774,Il y était écrit: Le bruit se répand parmi les...


# Sentence cutting isn't enough

We will now perform some kind of chunking to identfy problematic sentences

In [ ]:
# new filtering technique here